In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  

# Análise Exploratória dos Dados das Bombas
Este notebook tem como objetivo explorar, limpar e analisar os dados coletados das bombas, buscando entender padrões, identificar possíveis problemas e preparar os dados para análises futuras.

O fluxo do notebook inclui:
- Carregamento dos dados
- Explicação das colunas do dataset
- Limpeza e transformação dos dados
- Análise estatística e visualização
- Discussão de hipóteses e conclusões

In [ ]:
df_concat = pd.read_csv("../../../data/complete.csv", low_memory=False)

### Dicionário das Colunas do Dataset
Abaixo estão as principais colunas do dataset, com explicação e tipo de dado:

| Coluna        | Descrição                                        | Tipo         |
|--------------|---------------------------------------------------|--------------|
| timestamp    | Data e hora da medição                            | datetime     |
| motor_pump   | Identificador da bomba                             | string       |
| resource     | Tipo de recurso medido                            | string       |
| value        | Valor medido do recurso                           | float         |
| origin       | Origem do dado (broker)                           | string       |
| serial       | Número do gateway instalado na máquina            | string       |
| type         | Indica parte onde houve falha (quando aplicável)  | string       |
| data_consulta| Data arredondada da consulta                      | datetime     |

---

In [ ]:
df_concat = df_concat[::-1]

### Limpeza e Transformação dos Dados
Nesta etapa, realizamos a limpeza dos dados para garantir que estejam prontos para análise:
- Conversão da coluna `timestamp` para o formato de data e hora.
- Conversão da coluna `value` para tipo numérico (float).
- Remoção de colunas desnecessárias (`origin`, `serial`, `data_consulta`) dos dataframes filtrados.
- Verificação e tratamento de valores nulos e zeros, especialmente em recursos como RPM, onde valores zerados podem indicar máquina parada.
- Normalização dos valores para facilitar comparações entre variáveis de diferentes escalas.

In [ ]:
df_concat['timestamp'] = pd.to_datetime(df_concat['timestamp'])

In [ ]:
df_concat.head()

In [ ]:
df_concat.info()

In [ ]:
df_concat.head()

In [ ]:
len(df_concat)

In [ ]:
df_concat.shape

In [ ]:
df_concat['value'] = df_concat['value'].astype(float)

In [ ]:
df_concat['type'].value_counts()

In [ ]:
df_concat['resource'].value_counts()

In [ ]:
filtered_df_eng_rpm = df_concat[df_concat['resource'] == 'Eng_RPM']
filtered_df_bat_v = df_concat[df_concat['resource'] == 'Bat_V']
filtered_df_char_v = df_concat[df_concat['resource'] == 'Char_V']
filtered_df_oil_p = df_concat[df_concat['resource'] == 'Oil_P'] 

## Análise Estatística e Visualização
Após a limpeza, realizamos análises estatísticas e criamos gráficos para entender o comportamento das variáveis ao longo do tempo.

- Estatísticas descritivas (média, contagem, valores nulos, zeros) para cada recurso.
- Visualização das séries temporais de RPM, tensão da bateria, alternador e pressão do óleo.
- Comparação entre variáveis normalizadas para identificar possíveis correlações.

Essas análises ajudam a identificar padrões, comportamentos anormais e possíveis relações entre os diferentes sensores das bombas.

### Tipos de resources encontrados

ENG_RPM: Rotação do motor em RPM <br> 
BAT_V: Tensão da bateria em Volts <br>
CHAR_V: Tensão no alternador em Volts <br>
OIL_P: Pressão do óleo em MCA (metros de coluna d'água)

In [ ]:
filtered_df_eng_rpm.head()

In [ ]:
filtered_df_bat_v = filtered_df_bat_v.drop(columns=['origin', 'serial', 'data_consulta'], axis=1)
filtered_df_eng_rpm = filtered_df_eng_rpm.drop(columns=['origin', 'serial', 'data_consulta'], axis=1)
filtered_df_char_v = filtered_df_char_v.drop(columns=['origin', 'serial', 'data_consulta'], axis=1)
filtered_df_oil_p = filtered_df_oil_p.drop(columns=['origin', 'serial', 'data_consulta'], axis=1)

In [ ]:
filtered_df_bat_v.info()

In [ ]:
filtered_df_eng_rpm['value'] = filtered_df_eng_rpm['value'].astype('float') 
filtered_df_oil_p['value'] = filtered_df_oil_p['value'].astype('float')  
filtered_df_char_v['value'] = filtered_df_char_v['value'].astype('float')  
filtered_df_bat_v['value'] = filtered_df_bat_v['value'].astype('float')   

In [ ]:
filtered_df_char_v['value']

In [ ]:
filtered_df_eng_rpm.info()

In [ ]:
filtered_df_eng_rpm['value'].describe()

In [ ]:
filtered_df_eng_rpm.isna().sum()

In [ ]:
filtered_df_eng_rpm['motor_pump'].nunique()

In [ ]:
zeros = (filtered_df_eng_rpm['value'] == 0).sum()
zeros

In [ ]:
## 30.320 dados, de um total de 319.701, são diferentes de 0. Ou seja, só cerca de 10%.
filtered_df_eng_rpm.shape[0] - zeros

In [ ]:
percentage = (zeros * 100) / filtered_df_eng_rpm.shape[0]
print(f"Porcentagem de rpm zerado: {percentage:.2f}%")
print(f"Porcentagem de rpm diferente de zero: {100 - percentage:.2f}%")

In [ ]:
print(f"Média de RPM (excluindo zeros): {filtered_df_eng_rpm[filtered_df_eng_rpm["value"] != 0]["value"].mean():.2f}rpm")

In [ ]:
print(f"Média de tensao da bateria em volts (excluindo zeros): {filtered_df_bat_v[filtered_df_bat_v["value"] != 0]["value"].mean():.2f}V")

In [ ]:
print(f"Média de tensao do alternador em volts (excluigit@github.com:Inteli-College/2025-2A-T12-EC07-G03.gitndo zeros): {filtered_df_char_v[filtered_df_char_v["value"] != 0]["value"].mean():.2f}V")

In [ ]:
print(f"Média de pressão do óleo em mca (excluindo zeros): {filtered_df_oil_p[filtered_df_oil_p["value"] != 0]["value"].mean():.2f}mca")

# Gráficos

A seguir, apresentamos gráficos para visualizar o comportamento das variáveis principais ao longo do tempo:
- **RPM**: Mostra a rotação do motor, útil para identificar funcionamento e paradas.
- **Tensão da bateria**: Permite observar variações e possíveis quedas relacionadas ao funcionamento do motor.
- **Tensão do alternador**: Indica o desempenho do sistema de carga.
- **Pressão do óleo**: Ajuda a monitorar a saúde do motor e possíveis falhas.
- **Comparação entre variáveis**: Gráficos que mostram as variáveis normalizadas juntas para facilitar a identificação de correlações e padrões.


In [ ]:
start_date = '2025-07-05 08:56:15.183619+00:00'
period_in_hours = 48
motor_pump = 'ITU-678'

In [ ]:
start = pd.to_datetime(start_date)
end = start + pd.Timedelta(hours=period_in_hours) 

mascara_eng_rpm = (filtered_df_eng_rpm['timestamp'] >= start) & (filtered_df_eng_rpm['timestamp'] < end)
filtered_df_eng_rpm_with_especific_time = filtered_df_eng_rpm[mascara_eng_rpm]
filtered_df_eng_rpm_with_especific_time = filtered_df_eng_rpm_with_especific_time[filtered_df_eng_rpm_with_especific_time['motor_pump'] == motor_pump]

mascara_bat_v = (filtered_df_bat_v['timestamp'] >= start) & (filtered_df_bat_v['timestamp'] < end) 
filtered_df_bat_v_with_especific_time = filtered_df_bat_v[mascara_bat_v]
filtered_df_bat_v_with_especific_time = filtered_df_bat_v_with_especific_time[filtered_df_bat_v_with_especific_time['motor_pump'] == motor_pump]

mascara_oil_p = (filtered_df_oil_p['timestamp'] >= start) & (filtered_df_oil_p['timestamp'] < end)
filtered_df_oil_p_with_especific_time = filtered_df_oil_p[mascara_oil_p]
filtered_df_oil_p_with_especific_time = filtered_df_oil_p_with_especific_time[filtered_df_oil_p_with_especific_time['motor_pump'] == motor_pump]

mascara_char_v = (filtered_df_char_v['timestamp'] >= start) & (filtered_df_char_v['timestamp'] < end)
filtered_df_char_v_with_especific_time = filtered_df_char_v[mascara_char_v]
filtered_df_char_v_with_especific_time = filtered_df_char_v_with_especific_time[filtered_df_char_v_with_especific_time['motor_pump'] == motor_pump]

In [ ]:
filtered_df_eng_rpm_with_especific_time.shape

In [ ]:
plt.figure(figsize=(24, 12))
plt.subplot(2, 2, 1)

if filtered_df_eng_rpm_with_especific_time.shape[0] == 0:
    plt.text(0.5, 0.5, 'no data for the specified time window', fontsize=14, ha='center', va='center', transform=plt.gca().transAxes)
else:
    plt.plot(filtered_df_eng_rpm_with_especific_time['timestamp'], filtered_df_eng_rpm_with_especific_time['value'])

print(filtered_df_eng_rpm_with_especific_time.shape)
plt.xlabel('Timestamp')
plt.xlim(start, end)
plt.ylim(0, 2400)
plt.ylabel('RPM')
plt.title('Variação de RPM ao longo do tempo')
plt.tight_layout()

plt.subplot(2, 2, 2)

if filtered_df_bat_v_with_especific_time.shape[0] == 0:
    plt.text(0.5, 0.5, 'no data for the specified time window', fontsize=14, ha='center', va='center', transform=plt.gca().transAxes)

else:
    plt.plot(filtered_df_bat_v_with_especific_time['timestamp'], filtered_df_bat_v_with_especific_time['value'])

plt.xlabel('Timestamp')
plt.xlim(start, end)
plt.ylabel('V')
plt.title('Variação de tensão de bateria ao longo do tempo')
plt.tight_layout()
plt.ylim(20, 26)

plt.subplot(2, 2, 3)
if filtered_df_oil_p_with_especific_time.shape[0] == 0:
    plt.text(0.5, 0.5, 'no data for the specified time window', fontsize=14, ha='center', va='center', transform=plt.gca().transAxes)
else:
    plt.plot(filtered_df_oil_p_with_especific_time['timestamp'], filtered_df_oil_p_with_especific_time['value'])

plt.xlabel('Timestamp')
plt.xlim(start, end)
plt.ylabel('mca')
plt.title('Variação de pressão de óleo ao longo do tempo')
plt.tight_layout()

plt.subplot(2, 2, 4)
plt.plot(filtered_df_char_v_with_especific_time['timestamp'], filtered_df_char_v_with_especific_time['value'])
if filtered_df_char_v_with_especific_time.shape[0] == 0:
    plt.text(0.5, 0.5, 'no data for the specified time window', fontsize=14, ha='center', va='center', transform=plt.gca().transAxes)
else:
    plt.plot(filtered_df_oil_p_with_especific_time['timestamp'], filtered_df_oil_p_with_especific_time['value'])

plt.xlabel('Timestamp')
plt.xlim(start, end)
plt.ylabel('V')
plt.title('Variação de tensão do alternador ao longo do tempo')
plt.tight_layout()

plt.show()

In [ ]:
plt.figure(figsize=(24, 12))
plt.plot(filtered_df_eng_rpm_with_especific_time['timestamp'], filtered_df_eng_rpm_with_especific_time['value'])
plt.xlabel('Timestamp')
plt.ylabel('RPM')
plt.title('Variação de RPM ao longo do tempo')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(filtered_df_bat_v_with_especific_time['timestamp'], filtered_df_bat_v_with_especific_time['value'])
plt.xlabel('Timestamp')
plt.ylabel('V')
plt.title('Variação de tensão de bateria ao longo do tempo')
plt.tight_layout()
plt.ylim(20, 26)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(filtered_df_oil_p_with_especific_time['timestamp'], filtered_df_oil_p_with_especific_time['value'])
plt.xlabel('Timestamp')
plt.ylabel('mca')
plt.title('Variação de pressão de óleo ao longo do tempo')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(filtered_df_char_v_with_especific_time['timestamp'], filtered_df_char_v_with_especific_time['value'])
plt.xlabel('Timestamp')
plt.ylabel('V')
plt.title('Variação de tensão do alternador ao longo do tempo')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(filtered_df_eng_rpm_with_especific_time['timestamp'], filtered_df_eng_rpm_with_especific_time['value'], label='rpm')
plt.plot(filtered_df_bat_v_with_especific_time['timestamp'], filtered_df_bat_v_with_especific_time['value'], label='bateria')
plt.plot(filtered_df_oil_p_with_especific_time['timestamp'], filtered_df_oil_p_with_especific_time['value'], label='oleo')
plt.plot(filtered_df_char_v_with_especific_time['timestamp'], filtered_df_char_v_with_especific_time['value'], label='alternador')
plt.xlabel('Timestamp')
plt.tight_layout()
plt.show()

In [ ]:
## (valor - minimo) / (maximo - minimo) 

df_concat['normalized'] = (df_concat['value'] - df_concat['value'].min()) / (df_concat['value'].max() - df_concat['value'].min())

In [ ]:
rpm = df_concat[df_concat['resource'] == 'Eng_RPM']
oil = df_concat[df_concat['resource'] == 'Oil_P']
bat = df_concat[df_concat['resource'] == 'Bat_V']
alt = df_concat[df_concat['resource'] == 'Char_V']

In [ ]:
start = pd.to_datetime(start_date)
end = start + pd.Timedelta(hours=period_in_hours) 

mascara_eng_rpm = (rpm['timestamp'] >= start) & (rpm['timestamp'] < end)
rpm_timestamp = rpm[mascara_eng_rpm]
rpm_timestamp = rpm_timestamp[rpm_timestamp['motor_pump'] == motor_pump]

mascara_bat_v = (bat['timestamp'] >= start) & (bat['timestamp'] < end) 
bat_timestamp = bat[mascara_bat_v]
bat_timestamp = bat_timestamp[bat_timestamp['motor_pump'] == motor_pump]

mascara_oil_p = (oil['timestamp'] >= start) & (oil['timestamp'] < end)
oil_timestamp = oil[mascara_oil_p]
oil_timestamp = oil_timestamp[oil_timestamp['motor_pump'] == motor_pump]

mascara_char_v = (alt['timestamp'] >= start) & (alt['timestamp'] < end)
alt_timestamp = alt[mascara_char_v]
alt_timestamp = alt_timestamp[alt_timestamp['motor_pump'] == motor_pump]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(rpm_timestamp['timestamp'], rpm_timestamp['normalized'], label='rpm')
plt.plot(bat_timestamp['timestamp'], bat_timestamp['normalized'], label='bateria')
plt.plot(oil_timestamp['timestamp'], oil_timestamp['normalized'], label='oleo')
plt.plot(alt_timestamp['timestamp'], alt_timestamp['normalized'], label='alternador')
plt.xlabel('Timestamp')
plt.tight_layout()
plt.show()

trabalhos que foram feitos com dados de manutenção (uma forma melhor de olhar para esses dados ex: o que é feito com dados de rotação, etc)



escrever conclusões e formular hipoteses


# Conclusões
As principais conclusões da análise exploratória são:
- **Medições em momentos diferentes:** As variáveis não são medidas simultaneamente, o que dificulta, mas não impede, a correlação entre elas.
- **Rotação baixa indica problema:** Rotação do motor abaixo de 1400-1600 rpm pode indicar funcionamento inadequado.
- **Quedas bruscas:** Uma queda brusca é caracterizada por redução rápida ou zeramento da rotação em menos de 15-30 segundos.
- **Dados de falha:** As tabelas de alerta/falha possuem apenas os recursos relevantes para cada tipo de falha, o que pode ser útil para treinar modelos de predição. Porém, a falta de dados em momentos de normalidade na operação, dificultam o desenvolvimento de um modelo, pois ele só conhece os momentos de falha, gerando a falta de um ponto de referência.
- **Relação entre variáveis:** Não foi possível identificar claramente relações esperadas, como queda de tensão da bateria durante a partida do motor, nos gráficos gerados.

Por fim, é possível concluir que, com os dados disponibilizados, se torna difícil encontrar alguma relação e chegar a conclusões concretas, ou seja, enquanto os novos dados de operação não chegarem, não será possível encontrar correlações importantes para treinamento de um modelo.


# Hipóteses
A partir da análise dos dados, levantam-se as seguintes hipóteses:
- Variações pequenas de rpm (ex: 1100 para 1096) provavelmente não indicam problemas relevantes.
- Variações abruptas de tensão da bateria podem sinalizar falha ou comportamento anormal. Uma bateria de 24V pode ter tolerância de ±2V.
- Como os dados não são medidos simultaneamente, pode ser interessante agregar médias por intervalos (ex: 1 minuto) para comparar tendências.
- Mudanças abruptas de rpm em até 30s (diferença entre linhas) podem indicar eventos críticos.
- As variáveis mais importantes para monitoramento são: rotação, vazão de fluido e pressão.
- Para análises futuras, é importante relacionar dados de funcionamento normal e de falha, especialmente para bombas presentes em diferentes tabelas.

As hipóteses levantadas orientam os próximos passos da análise, como o desenvolvimento de modelos preditivos e a busca por padrões de falha. A normalização e o agrupamento dos dados serão essenciais para extrair informações relevantes e apoiar decisões de manutenção preventiva.